In [1]:
import tensorflow as tf
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model
from tensorflow import keras
from tensorflow.keras import backend as kb
from IPython.display import clear_output

In [2]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
# Create a dictionary describing the features.
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'image_name': tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.string)
}
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

def features_preprocess(features):
    
    # Image
    image_raw = features['image']
    image_ = tf.io.decode_image(image_raw)
    image = tf.cast(image_ , tf.float32) * (1. / 255)
    
    # Image name
    image_name_raw = features['image_name'].numpy().decode()
    image_name = image_name_raw.split('_')[0]
    image_color = image_name_raw.split('_')[1]
    
    # Image Label
    target_raw = features['target'].numpy().decode()
    target_list = target_raw.split('|')

    return image, image_name, image_color, target_list

In [ ]:
test_files = os.listdir('./test_tfrecords')
test_files.sort()

train_files = os.listdir('./train_tfrecords')
train_files.sort()

dataset = None
dataset_batched = None
trainDataset = None
validDataset = None

trial = 200

for trial_ in range(trial):
    selected = np.random.choice(len(train_files), 1, replace=False)
    
    count_positive = 0
    inputs_positive = np.empty([1, 512, 512, 4])
    outputs_positive = np.empty([1, 1])
    
    count_negative = 0
    inputs_negative = np.empty([1, 512, 512, 4])
    outputs_negative = np.empty([1, 1])
    
    ratio_positive = np.random.uniform(low=0.3, high=0.7)
    
    buffer = {}
    current_file = ''
    
    for idx in selected:
        dir_ = os.path.join('train_tfrecords', train_files[idx])
        
        raw_dataset = tf.data.TFRecordDataset(dir_)
        parsed_dataset = raw_dataset.map(_parse_image_function)
        
        for features in parsed_dataset:
            image, image_name, image_color, target_list = features_preprocess(features)

            if current_file != image_name:
                buffer = {}
                current_file = image_name

            buffer[image_color] = image

            # 하나의 이미지에 대한 4개의 색깔 이미지가 다 보인 시점
            if len(buffer.keys()) == 4:

                if '0' in target_list:
                    
                    count_positive += 1
                    input_ = tf.expand_dims(tf.concat([buffer['blue'], buffer['yellow'], buffer['green'], buffer['red'] ], 2), 0)
                    input_ = input_[:, 256:768, 256:768, :]
                    output_ = tf.constant([[1]])

                    inputs_positive = np.concatenate((inputs_positive, input_), axis=0)
                    outputs_positive = np.concatenate((outputs_positive, output_), axis=0)


                else:
                    count_negative += 1
                    input_ = tf.expand_dims(tf.concat([buffer['blue'], buffer['yellow'], buffer['green'], buffer['red'] ], 2), 0)
                    input_ = input_[:, 256:768, 256:768, :]
                    output_ = tf.constant([[0]])

                    inputs_negative = np.concatenate((inputs_negative, input_), axis=0)
                    outputs_negative = np.concatenate((outputs_negative, output_), axis=0)
    
    
    
    inputs_positive = np.delete(inputs_positive, 0, axis=0)
    outputs_positive = np.delete(outputs_positive, 0, axis=0)
    
    inputs_negative = np.delete(inputs_negative, 0, axis=0)
    outputs_negative = np.delete(outputs_negative, 0, axis=0)
    
    if count_positive / ratio_positive * (1 - ratio_positive) < count_negative:
        choise_ = np.random.choice(count_negative, int(count_negative * (1 - ratio_positive)), replace=False)
        inputs_negative = inputs_negative[choise_,:,:,:]
        outputs_negative = outputs_negative[choise_,:]
        
        
        
    else:
        
        choise_ = np.random.choice(count_positive, int(count_positive * ratio_positive), replace=False)
        inputs_positive = inputs_positive[choise_,:,:,:]
        outputs_positive = outputs_positive[choise_,:]
        
        
        
    inputs = np.concatenate((inputs_positive, inputs_negative), axis=0)
    outputs = np.concatenate((outputs_positive, outputs_negative), axis=0)
    dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs)).shuffle(inputs.shape[0], reshuffle_each_iteration=True)
    dataset_batched = dataset.batch(16, drop_remainder=True)
    numberOfData = dataset_batched.cardinality().numpy()
    trainDataset = dataset_batched.take(int(numberOfData * 0.7))
    validDataset = dataset_batched.skip(int(numberOfData * 0.7))
    
    
    train_session(trial_+1)

Trial_num: 9, 에포크: 262, 손실: 0.4879, 정확도: 80.196, 테스트 손실: 0.4875, 테스트 정확도: 80.047, 소요시간: 3.325


In [4]:
class MyModel(Model):
    def __init__(self, list_units):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, padding='same', activation='swish')
        self.pool1 = MaxPooling2D()
        
        self.conv2 = Conv2D(32, 3, padding='same', activation='swish')
        self.pool2 = MaxPooling2D()
        
        self.conv3 = Conv2D(32, 3, padding='same', activation='swish')
        self.pool3 = MaxPooling2D()
        
        self.flatten = Flatten()
        
        self.layers_custom = self.create_layers(list_units)
        self.lastLayer = Dense(1, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(0.001))
        
    def create_layers(self, list_units):
        layers = []
        for units in list_units:
            layers.append(Dense(units, activation='swish', kernel_regularizer=keras.regularizers.l2(0.001)))
        return layers

    def call(self, x):
        
        x = self.conv1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.pool3(x)
        
        x = self.flatten(x)
        
        for layer in self.layers_custom:
            x = layer(x)
        x = self.lastLayer(x)
        
        return x

model = MyModel([7, 7])
#model.load_weights('./kaggle_models/class0_77_file1')


In [5]:

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)

train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.BinaryAccuracy()

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        train_loss(loss)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_acc.update_state(labels, predictions)

    
    
valid_loss = tf.keras.metrics.Mean()
valid_acc = tf.keras.metrics.BinaryAccuracy()

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    valid_loss(t_loss)
    valid_acc.update_state(labels, predictions)

In [6]:
def train_session(trial_number):
    
    global dataset
    global dataset_batched
    global trainDataset
    global validDataset
    
    EPOCHS = 1000
    patience = 100
    stopped_epoch = 0
    best_weights = None
    best = np.Inf
    wait = 0


    history_trainLoss = []
    history_validLoss = []

    for epoch in range(EPOCHS):
        clear_output(wait=True)
        start_time = time.time()

        for images, labels in trainDataset:
            train_step(images, tf.expand_dims(labels, axis=-1))
        for images, labels in validDataset:
            test_step(images, tf.expand_dims(labels, axis=-1))

        template = 'Trial_num: {}, 에포크: {}, 손실: {:.4f}, 정확도: {:.3f}, 테스트 손실: {:.4f}, 테스트 정확도: {:.3f}, 소요시간: {:.3f}'
        print (template.format(trial_number,
                               epoch+1,
                             train_loss.result(),
                             train_acc.result()*100,
                             valid_loss.result(),
                             valid_acc.result()*100,
                               time.time()-start_time))

        history_trainLoss.append(train_loss.result())
        history_validLoss.append(valid_loss.result())

        if np.less(float(valid_loss.result()), best):
            best = float(valid_loss.result())
            best_weights = model.get_weights()
            wait = 0
        else:
            wait +=1
            if wait >= patience:
                model.set_weights(best_weights)
                stopped_epoch = epoch
                print('Early Stopped !')
                break



    fig= plt.figure(figsize=(18,20))
    plt.title('Loss')
    x = np.arange(len(history_trainLoss))
    plt.plot(x, history_trainLoss, 'r-', label = 'history_trainLoss')
    plt.plot(x, history_validLoss, 'b-', label = 'history_validLoss')
    plt.legend()

    model.save_weights('./kaggle_models/trial_' + str(trial_number))
    
    dataset = None
    dataset_batched = None
    trainDataset = None
    validDataset = None

In [48]:

trial = 100

for i in range(trial):
    np.random.choice(len(train_files), 2, replace=False)

In [10]:
tmp1 = tf.constant(1, shape=(3, 2, 1))
tmp2 = tf.constant(1, shape=(3, 2, 1))
print(tf.concat([tmp1, tmp2], 2).shape)
    

(3, 2, 2)


0.31551865344251245